<a href="https://colab.research.google.com/github/studam/Big-Data/blob/main/Level2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpa

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-04-27 19:59:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.11MB/s    in 0.8s    

2022-04-27 19:59:53 (1.11 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("big_data_assignment_level_1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [15]:
#pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_df = spark.read.csv(SparkFiles.get(""), sep="\t", header=True)
watches_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [16]:
#pyspark
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Watches_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
watches_df = spark.read.csv(SparkFiles.get(""), sep="\t", header=True)
watches_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    3653882|R3O9SGZBVQBV76|B00FALQ1ZC|     937001370|Invicta Women's 1...|         Watches|          5|            0|          0|   N|                Y|          Five Stars|Absolutely love t...| 2015-08-31|
|         US|   14661224| RKH8BNC3L5DLF|B00D3RGO20|     484010722|Kenneth Cole New ...|         Watches|          5|    

In [17]:
#pyspark
# Count rows
watches_df.count()

960872

In [18]:

# Filter by voters.
df2 = watches_df.select(["star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
df2.show(10)

df3 = df2.filter(df2['total_votes'] >= 20)
df4 = df3.filter(df3["helpful_votes"]/df3["total_votes"] >= 0.5)

+-----------+-------------+-----------+----+-----------------+
|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+-----------+-------------+-----------+----+-----------------+
|          5|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          2|            1|          1|   N|                Y|
|          5|            0|          0|   N|                Y|
|          4|            0|          0|   N|                Y|
|          5|            0|          0|   N|                Y|
|          5|            1|          1|   N|                Y|
|          1|            1|          5|   N|                N|
|          5|            1|          2|   N|                Y|
|          4|            0|          0|   N|                Y|
+-----------+-------------+-----------+----+-----------------+
only showing top 10 rows



In [19]:

# Describe stats.
from pyspark.sql.functions import col, avg
paid_df = df4.filter(df4['vine']== 'Y')
unpaid_df = df4.filter(df4['vine']== 'N')

paid_df.describe().show()
unpaid_df.describe().show()

+-------+------------------+-----------------+-----------------+----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+-----------------+-----------------+----+-----------------+
|  count|                47|               47|               47|  47|               47|
|   mean|3.9148936170212765|42.61702127659574|47.91489361702128|null|             null|
| stddev|1.0597330926073165|51.83441579201542|55.06629592837206|null|             null|
|    min|                 1|              107|              121|   Y|                N|
|    max|                 5|               81|               92|   Y|                N|
+-------+------------------+-----------------+-----------------+----+-----------------+

+-------+------------------+-----------------+-----------------+----+-----------------+
|summary|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|
+-------+------------------+---

In [20]:
#  Determine the percentage of five-star reviews among Vine reviews
paid_five_star_number = paid_df[paid_df['star_rating']== 5].count()
paid_number = paid_df.count()
percentage_five_star_vine = float(paid_five_star_number) / float(paid_number)
print(paid_number)
print(paid_five_star_number)
print(percentage_five_star_vine)

47
15
0.3191489361702128


In [21]:

#  Determine the percentage of five-star reviews among non-Vine reviews.
unpaid_five_star_number = unpaid_df[unpaid_df['star_rating']== 5].count()
unpaid_number = unpaid_df.count()
percentage_five_star_non_vine = float(unpaid_five_star_number) / float(unpaid_number)
print(unpaid_number)
print(unpaid_five_star_number)
print(float(unpaid_five_star_number) / float(unpaid_number))

8362
4332
0.5180578808897393
